In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import category_encoders as ce


In [ ]:
df = pd.read_csv('../dataset/weatherAUS.csv')

In [ ]:
print('Broj redova: {}'.format(df.shape[0]))
print('Broj kolona:{}'.format(df.shape[1]))

In [ ]:
df.describe(include='all')

In [ ]:
print(df['Date'].min())
print(df['Date'].max())
print(df.columns)

In [ ]:
kategoricki = [col for col in df.columns if df[col].dtype=="object" ]
numericki = [col for col in df.columns if df[col].dtype!="object" ]

print("{} kategorickih :".format(len(kategoricki)))
print(kategoricki)

print("\n{} numerickih :".format(len(numericki)))
print(numericki)


Nedostajuce vrednosti

In [ ]:
nv=pd.concat([df.isnull().sum(), 100 * df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
print(nv[nv.Missing_Records>0].sort_values('Missing_Records', ascending=False))
pl = plt.pie(nv[nv.Missing_Records>0]['Missing_Records'],labels = nv[nv.Missing_Records>0].axes[0],radius=2.5,labeldistance=1.2)

#plt.pie(df.isnull().sum())


In [ ]:
print("{} nedostajucih vrednosti za cilj".format(df['RainTomorrow'].isnull().sum()))

In [ ]:
df = df[df['RainTomorrow'].notna()]
df.shape

In [ ]:
df['RainTomorrow'].hist(xlabelsize=15,bins=3,legend=True)


Vidimo da nije ravnomerno rasporedjeno. Balansiranje cemo se baviti u sklopu algoritama za klasifikaciju

In [ ]:
sns.heatmap(df.isnull())

Najvise nedostajucih vrednosti u Evaporation, Sunshine, Cloud9am and Cloud3pm. Ne mozemo da izbacimo redove koji sadrze nepoznate vrednosti.


In [ ]:
print(df['Evaporation'].mean())
print(df['Sunshine'].mean())
print(df['Cloud3pm'].mean())
print(df['Cloud9am'].mean())

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(),cmap='RdYlGn', annot=True)


In [ ]:
def max_corr(column):
    print('Srednja vrednost pre dodavanja nepoznatih vrednosti {}'.format(df[column].mean()))
    index = df.corr().columns.get_loc(column)
    print(index)
    n = len(df.corr())
    corr = []
    max_corr = 0
    index_max = -1
    for i in range(n):
        if i == index:
            continue
        curr_corr = abs(df.corr().iloc[[index]].iloc[[0],[i]].max()[0])
        
        corr.append((curr_corr,i))
        
    corr.sort(reverse=True)
    max_cor = [corr[0][0],corr[1][0],corr[2][0]]
    attributes = [df.corr().columns[corr[0][1]],df.corr().columns[corr[1][1]],df.corr().columns[corr[2][1]]]
    print('Najvece korelacije {}'.format(max_cor))
    print('Atributi: {}'.format(attributes))
    print("Histogram pre dodavanja vrednosti:")
    plt.hist(df[column])
    return attributes


Evaporation

In [ ]:
atr = max_corr('Evaporation')


     Vidimo da ima najvecu korelaciju sa atributima MaxTemp,Temp3pm i Temp9am

In [ ]:
print(df['Evaporation'].ffill().mean())
mean =df.sort_values(by="MaxTemp")["Evaporation"].backfill().mean()
print(mean)
mean =df.sort_values(by=atr)["Evaporation"].backfill().mean()
print(mean)
df = df.sort_values(by=atr)
df["Evaporation"] = df["Evaporation"].backfill()
print("Histogram posle dodavanja vrednosti:")
plt.hist(df['Evaporation'])


Sunshine

In [ ]:
atr = max_corr('Sunshine')


Vidimo da ima najvecu korelaciju sa atributom Cloud3pm, CLoud9am, Humidity3pm

In [ ]:
print(df['Sunshine'].ffill().mean())
mean =df.sort_values(by=atr)["Sunshine"].backfill().mean()
print(mean)
df = df.sort_values(by=atr)
df["Sunshine"] = df["Sunshine"].backfill()
print("Histogram posle dodavanja vrednosti:")
plt.hist(df['Sunshine'])

Cloud9am

In [ ]:
atr = max_corr('Cloud9am')

In [ ]:
print(df['Cloud9am'].ffill().mean())
mean =df.sort_values(by=atr)["Cloud9am"].backfill().mean()
print(mean)
df = df.sort_values(by=atr)
df["Cloud9am"] = df["Cloud9am"].backfill()
print("Histogram posle dodavanja vrednosti:")
plt.hist(df['Cloud9am'])

Cloud3pm

In [ ]:
atr = max_corr('Cloud3pm')

In [ ]:
print(df['Cloud3pm'].ffill().mean())
mean =df.sort_values(by=atr)["Cloud3pm"].backfill().mean()
print(mean)
df = df.sort_values(by=atr)
df["Cloud3pm"] = df["Cloud3pm"].backfill()
print("Histogram posle dodavanja vrednosti:")
plt.hist(df['Cloud3pm'])

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
df.isna().sum()

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
for i in range(len(df.corr())):
    name = df.corr().columns[i]
    df[name] = df[name].fillna(df[name].mean())

In [ ]:
sns.heatmap(df.isnull())

Kategoricki atributi.
Nedostajuce vrednosti cemo zameniti najcescim pojavljivanjem 

In [ ]:
category_columns = [col for col in df.columns if df[col].dtype=="object" ]
for col in category_columns:
     df[col].fillna(df[col].mode()[0], inplace=True)
        

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
df.shape

Enkodiranje

In [ ]:
df['Date']= pd.to_datetime(df['Date']) 
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day



#Dropping the date column
df.drop('Date', axis=1, inplace = True)

df.replace('Yes',1, inplace=True)
df.replace('No', 0, inplace=True)


In [ ]:
df.describe()

In [ ]:
print(df['WindDir3pm'].unique())
print(df['WindDir9am'].unique())
print(df['WindGustDir'].unique())
print(df['Location'].unique())

In [ ]:
mapping = {'N':1, 'NNE':2, 'NE':3,'ENE':4, 'E':5,'ESE':6,'SE':7,'SSE':8, 'S':9,'SSW':10,'SW':11,'WSW':12,'W':13,'WNW':14,
          'W':15,'WNW':16,'NW':17,'NNW':18}

df['WindDir3pm'] =df['WindDir3pm'].map(mapping)
df['WindDir9am'] = df['WindDir9am'].map(mapping)
df['WindGustDir'] =df['WindGustDir'].map(mapping)

print(df['WindDir3pm'].unique())
print(df['WindDir9am'].unique())
print(df['WindGustDir'].unique())
print(df['Location'].unique())

feature_names = df.columns
df.shape

In [ ]:
encoder = ce.TargetEncoder(cols=['Location'])
data_encoded = encoder.fit_transform(df['Location'],df['RainToday'])
df['Location'] = data_encoded

In [ ]:
print(df['Location'].unique())


#moguce preprilagodjavanje

In [ ]:
df.drop('Day', axis=1,inplace=True)
df.drop('Year',axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(),cmap='RdYlGn', annot=True)


Feature selection.
Izbacujemo atribure koji imaju veliku medjusobnu korelaciju

In [ ]:
threshold = 0.70
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
df = df.drop(df[to_drop], axis=1)

In [ ]:
print(df.shape)
df.columns

In [ ]:
df.to_csv("../dataset/weatherClean.csv",index=False)